In [7]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [8]:
def write_string_to_file(filename, content):
    try:
        with open(filename, 'w') as file:
            file.write(content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [9]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Choose important cases to generate, important scenario is the scenario that users are more likely to encounter it.
- Limit the appearance of scenarios that are hard to happen. 
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and corresponding use case.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

In [10]:
usecase_name = "Edit personal shopping cart"
project_name = "ShopeeWeb"
main_flow_prompt = """
Main flow
Step 1: User clicks on the shopping cart icon at the top right of the screen.
Step 2: System navigates user to the cart's page. There should be all the products that user have added but haven't bought yet and their corresponding information about which shop sells that product, the product's price, promotions information, quantity of the product that user have added and which option of the product the user have chosen if that product has many options.
"""
prompt_all = """
Main flow
Step 1: User clicks on the shopping cart icon at the top right of the screen.
Step 2: System navigates user to the cart's page. There should be all the products that user have added but haven't bought yet and their corresponding information about which shop sells that product, the product's price, promotions information, quantity of the product that user have added and which option of the product the user have chosen if that product has many options.

Alternative flow 1: User remove a product from their shopping cart by erasing it
Step 3: User ticks the box choosing the product they want to remove.
Step 4: User clicks at the Erase button at the bottom of the screen.
Step 5: System shows a pop up to confirm the changes with two options Yes or No.
Step 6: User clicks Yes.
Step 7: System removes the product from user's shopping cart.

Alternative flow 2: User remove a product from their shopping cart by erasing it but change mind
Step 3: User ticks the box choosing the product they want to remove.
Step 4: User clicks at the Erase button at the bottom of the screen.
Step 5: System shows a pop up to confirm the changes with two options Yes or No.
Step 6: User clicks No.

Alternative flow 3: User remove a product from their shopping cart by decreasing the quantity to zero
Step 3: User decreases the quantity of the product until it reached one by clicking the minus button in the quantity section.
Step 4: User clicks the minus button when the quantity is one.
Step 5: System shows a pop up to confirm the changes with two options Yes or No.
Step 6: User clicks Yes.
Step 7: System removes the product from user's shopping cart.

Alternative flow 4: User remove a product from their shopping cart by decreasing the quantity to zero
Step 3: User decreases the quantity of the product until the number reached one by clicking the minus button in the quantity section.
Step 4: User clicks the minus button when the quantity is one.
Step 5: System shows a pop up to confirm the changes with two options Yes or No.
Step 6: User clicks No.

Alternative flow 5: User increase a product's quantity
Step 3: User increases the quantity of the product until the number reachs what user want by clicking the plus button in the quantity section.

Alternative flow 6: User decrease a product's quantity
Step 3: User decreases the quantity of the product until the number reachs what user want by clicking the plus button in the quantity section.

Alternative flow 7: User change options of a product
Step 3: User clicks on the option dropdown of a product in shopping cart's list
Step 4: User clicks on the option they want to change to.
Step 5: System changes the option of that product to what user has chosen. 
"""
alt_prompt=[
"""Alternative flow 1: User remove a product from their shopping cart by erasing it
Step 3: User ticks the box choosing the product they want to remove.
Step 4: User clicks at the Erase button at the bottom of the screen.
Step 5: System shows a pop up to confirm the changes with two options Yes or No.
Step 6: User clicks Yes.
Step 7: System removes the product from user's shopping cart.""",
"""Alternative flow 2: User remove a product from their shopping cart by erasing it but change mind
Step 3: User ticks the box choosing the product they want to remove.
Step 4: User clicks at the Erase button at the bottom of the screen.
Step 5: System shows a pop up to confirm the changes with two options Yes or No.
Step 6: User clicks No.""",
"""Alternative flow 3: User remove a product from their shopping cart by decreasing the quantity to zero
Step 3: User decreases the quantity of the product until it reached one by clicking the minus button in the quantity section.
Step 4: User clicks the minus button when the quantity is one.
Step 5: System shows a pop up to confirm the changes with two options Yes or No.
Step 6: User clicks Yes.
Step 7: System removes the product from user's shopping cart.
""",
"""Alternative flow 4: User remove a product from their shopping cart by decreasing the quantity to zero
Step 3: User decreases the quantity of the product until the number reached one by clicking the minus button in the quantity section.
Step 4: User clicks the minus button when the quantity is one.
Step 5: System shows a pop up to confirm the changes with two options Yes or No.
Step 6: User clicks No.""",
"""Alternative flow 5: User increase a product's quantity
Step 3: User increases the quantity of the product until the number reachs what user want by clicking the plus button in the quantity section.
""",
"""Alternative flow 6: User decrease a product's quantity
Step 3: User decreases the quantity of the product until the number reachs what user want by clicking the plus button in the quantity section.
""",
"""Alternative flow 7: User change options of a product
Step 3: User clicks on the option dropdown of a product in shopping cart's list
Step 4: User clicks on the option they want to change to.
Step 5: System changes the option of that product to what user has chosen. """]
exc_prompt = []

In [11]:
for i in range(1,4):
    print(f"D:\\Dissertation-GPT\\evaluate\\ResultSet\\5.20\\old filter\\{project_name}\\{usecase_name}-{i}.txt")

D:\Dissertation-GPT\evaluate\ResultSet\5.20\old filter\ShopeeWeb\Edit personal shopping cart-1.txt
D:\Dissertation-GPT\evaluate\ResultSet\5.20\old filter\ShopeeWeb\Edit personal shopping cart-2.txt
D:\Dissertation-GPT\evaluate\ResultSet\5.20\old filter\ShopeeWeb\Edit personal shopping cart-3.txt


In [12]:

for i in range(3,4):
  print("TIME "+str(i))
  promptMainScenario = [
      { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
      { "role": "user", "content": main_flow_prompt}
    ]
  main_gpt_response = ask(promptMainScenario, client, model)
  filtercontent = main_gpt_response
  print(filtercontent)
  for alt in alt_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n\n"+alt}
      ]
    print(main_flow_prompt +"\n\n"+alt)
    sub_gpt_response = ask(promptSubScenario, client, model)
    filtercontent+=  "\n\n" + sub_gpt_response
    print(filtercontent)
  for exc in exc_prompt:
    promptSubScenario = [
        { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
        { "role": "user", "content": main_flow_prompt +"\n\n"+exc}
      ]
    print(main_flow_prompt +"\n\n"+exc)
    sub_gpt_response = ask(promptSubScenario, client, model)
    filtercontent+=  "\n\n" + sub_gpt_response
    print(filtercontent)

  promptExtractCondition = [
      { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
      { "role": "user", "content": prompt_all}
    ]
  gpt_response = askJSON(promptExtractCondition, client, model)
  full_elements = json.loads(gpt_response)
  condition_element = {key: value for key, value in full_elements.items() 
                                if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
  print(condition_element)
  if(len(condition_element)!= 0):
    promptScenarioForCondition = [
        { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
        { "role": "user", "content": prompt_all + '\n Element:' + str(condition_element)}
      ]
    condition_scenario_response = ask(promptScenarioForCondition, client, model)
    print(condition_scenario_response)
    filtercontent += "\n" + condition_scenario_response

  print(filtercontent)

  promptFilter = [
      { "role": "system", "content": FILTER_SYSTEM_PROMPT},
      { "role": "user", "content": filtercontent}
    ]
  filter_gpt_response = ask(promptFilter, client, model)
  print("filter_gpt_response \n" +filter_gpt_response)
  write_string_to_file(f"D:\\Dissertation-GPT\\evaluate\\ResultSet\\5.20\\old filter\\{project_name}\\{usecase_name}-{i}.txt", filter_gpt_response)

TIME 3
Based on the provided main flow, here are the scenarios that can happen:

1. Cart Display with Multiple Products
2. Cart Display with a Single Product
3. Cart Display with Products from Multiple Shops
4. Cart Display with Products Having Multiple Options Selected
5. Cart Display with Products Having Promotions
6. Empty Cart Display
7. Cart Display with Products but No Available Promotions

Main flow
Step 1: User clicks on the shopping cart icon at the top right of the screen.
Step 2: System navigates user to the cart's page. There should be all the products that user have added but haven't bought yet and their corresponding information about which shop sells that product, the product's price, promotions information, quantity of the product that user have added and which option of the product the user have chosen if that product has many options.


Alternative flow 1: User remove a product from their shopping cart by erasing it
Step 3: User ticks the box choosing the product they